# Generate towers
The purpose of this notebook is to generate a set of towers, evaluate them according to basic metrics and to save them to disk.

In [ ]:
import tower_generator

import tqdm

import pickle

import math

import numpy as np
import matplotlib.pyplot as plt

from utils.blockworld_library import bl_silhouette2_default,bl_nonoverlapping_simple
from utils.blockworld import *

from model.BFS_Lookahead_Agent import BFS_Lookahead_Agent
from model.BFS_Agent import BFS_Agent
from model.Astar_Agent import Astar_Agent

In [ ]:
generator = tower_generator.TowerGenerator(8,8,
                                        block_library=bl_nonoverlapping_simple,
                                        seed=42,
                                        padding=(2,0)
                                        )

In [ ]:
NUM_TOWERS  = 100
towers = []
for i in tqdm.tqdm(range(NUM_TOWERS)):
    towers.append(generator.generate())

In [ ]:
# look at towers
def visualize_towers(towers):
    fig,axes = plt.subplots(math.ceil(len(towers)/5),5,figsize=(20,20*math.ceil(len(towers)/25)))
    for axis, tower in zip(axes.flatten(),towers):
        axis.imshow(tower['bitmap']*1.0)
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_towers(towers)

In [ ]:
def get_tower_cost(agent,tower):
    cost = 0
    world = Blockworld(silhouette=tower['bitmap'],
                       block_library=tower['block_library'],
                       physics=True,
                       legal_action_space=True)
    agent.set_world(world)
    while world.status()[0] == 'Ongoing':
        _,step_info = agent.act()
        cost += step_info['states_evaluated']
    return cost,world.status()
    

In [ ]:
agent = BFS_Agent(random_seed=42)

In [ ]:
agent = Astar_Agent(random_seed=42)

In [ ]:
get_tower_cost(agent,towers[0])

In [ ]:
def block_to_dict(block):
    return {"x": block.x, "y": block.y, "width": block.width, "height": block.height}


In [ ]:
for tower in tqdm.tqdm(towers):
    cost,status = get_tower_cost(agent,tower)
    tower['cost'] = cost
    tower['success'] = status[0] == 'Win'

In [ ]:
# pickle the towers
pickle.dump(towers,open('generated_evaluated_towers.pkl','wb'))

In [ ]:
costs = [c['cost'] for c in towers]

In [ ]:
plt.hist(costs,
            bins=3000,
)
plt.xlim(0,100000)
plt.show()